In [64]:
import sklearn
sklearn.__version__

'0.22.1'

In [78]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


K_foldCV = pd.read_csv('HW2.csv')


In [79]:
K_foldCV.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [80]:
K_foldCV.select_dtypes(include='number').head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40


# Data preprocessing

## 查看資料型態

In [81]:
dtype_df = K_foldCV.dtypes.reset_index()
dtype_df

,index,0
0,age,int64
1,workclass,object
2,fnlwgt,int64
3,education,object
4,education_num,int64
5,marital_status,object
6,occupation,object
7,relationship,object
8,race,object
9,sex,object


In [82]:
K_foldCV.nunique()

age                  73
workclass             9
fnlwgt            21648
education            16
education_num        16
marital_status        7
occupation           15
relationship          6
race                  5
sex                   2
capital_gain        119
capital_loss         92
hours_per_week       94
native_country       42
income                2
dtype: int64

## 空值處理

In [83]:
K_foldCV = K_foldCV.replace(" ?", "None")

## 類別型欄位處理

In [84]:
K_foldCV.select_dtypes(include='object').head()

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,income
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


### 1. 對eduction & education_num做群聚編碼

In [85]:
K_foldCV['education'].value_counts().sort_index()

 10th              933
 11th             1175
 12th              433
 1st-4th           168
 5th-6th           333
 7th-8th           646
 9th               514
 Assoc-acdm       1067
 Assoc-voc        1382
 Bachelors        5355
 Doctorate         413
 HS-grad         10501
 Masters          1723
 Preschool          51
 Prof-school       576
 Some-college     7291
Name: education, dtype: int64

In [86]:
K_foldCV['education_num'].value_counts().sort_index()

1        51
2       168
3       333
4       646
5       514
6       933
7      1175
8       433
9     10501
10     7291
11     1382
12     1067
13     5355
14     1723
15      576
16      413
Name: education_num, dtype: int64

In [89]:
K_foldCV['education'] = K_foldCV['education'].fillna('None')
mean_df = K_foldCV.groupby(['education'])['education_num'].mean().reset_index()
mean_df.rename(columns ={"education_num" : "education_median" })

K_foldCV = pd.merge(K_foldCV, mean_df, how='left', on=['education'])
K_foldCV = K_foldCV.drop(['education'] , axis=1)
K_foldCV.head()


KeyError: 'education'

In [90]:
K_foldCV['workclass'].value_counts().sort_index() # none 欄位蠻多的仍視為一種類別

 Federal-gov           960
 Local-gov            2093
 Never-worked            7
 Private             22696
 Self-emp-inc         1116
 Self-emp-not-inc     2541
 State-gov            1298
 Without-pay            14
None                  1836
Name: workclass, dtype: int64

## 數值型資料處理

### 1. "capital_gain" & "capital_loss"

In [91]:
K_foldCV.iloc[:,9:11].describe()# 發現資料分布不平均

,capital_gain,capital_loss
count,32561.000000,32561.000000
mean,1077.648844,87.303830
std,7385.292085,402.960219
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,99999.000000,4356.000000


In [92]:
K_foldCV['capital_gain'].value_counts().sort_index()

0        29849
114          6
401          2
594         34
914          8
         ...  
25236       11
27828       34
34095        5
41310        2
99999      159
Name: capital_gain, Length: 119, dtype: int64

In [93]:
K_foldCV['capital_loss'].value_counts().sort_index()

0       31042
155         1
213         4
323         3
419         3
        ...  
3004        2
3683        2
3770        2
3900        2
4356        3
Name: capital_loss, Length: 92, dtype: int64

### 以百分位數95來替代極端值-Capital_gain

In [94]:
K_foldCV[K_foldCV['capital_gain'] == K_foldCV["capital_gain"].max()] = np.percentile(K_foldCV[~K_foldCV["capital_gain"].isnull()]['capital_gain'], q = 95)

In [95]:
q_all =[np.percentile(K_foldCV[~K_foldCV['capital_gain'].isnull()]['capital_gain'],q = i) for i in range(101)] 

q_100_df = pd.DataFrame({'q': list(range(101)),
              'value': q_all})

In [96]:
q_95 = q_100_df.iloc[95,1]

K_foldCV.loc[~(K_foldCV["capital_gain"]==0),'capital_gain'] = q_95

In [97]:
K_foldCV['capital_gain'].value_counts().sort_index()

0.0       29849
5013.0     2712
Name: capital_gain, dtype: int64

### 以百分位數95來替代極端值-Capital loss

In [98]:
q_all_2 =[np.percentile(K_foldCV[~K_foldCV['capital_loss'].isnull()]['capital_loss'],q = i) for i in range(101)] 

q_100_df_2 = pd.DataFrame({'q': list(range(101)),
              'value': q_all_2})

In [99]:
q_95 = q_100_df_2.iloc[95,1]

K_foldCV.loc[~(K_foldCV["capital_loss"]==0),'capital_loss'] = q_95

In [100]:
K_foldCV['capital_loss'].value_counts().sort_index()

0.0       30883
1340.0     1678
Name: capital_loss, dtype: int64

In [101]:
K_foldCV.iloc[:,9:11].describe()

,capital_gain,capital_loss
count,32561.000000,32561.000000
mean,417.531894,69.055619
std,1385.212414,296.257567
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,5013.000000,1340.000000


In [102]:
K_foldCV.head(5)

,age,workclass,fnlwgt,education_num_x,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,education_num_y
0,39.0,State-gov,77516.0,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,5013.0,0.0,40.0,United-States,<=50K,13.0
1,50.0,Self-emp-not-inc,83311.0,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,13.0
2,38.0,Private,215646.0,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,9.0
3,53.0,Private,234721.0,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,7.0
4,28.0,Private,338409.0,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,13.0


##  2."fnlwgt" 處理

In [103]:
K_foldCV['fnlwgt'].value_counts().sort_index()

5013.0       159
12285.0        1
13769.0        1
14878.0        1
18827.0        1
            ... 
1226583.0      1
1268339.0      1
1366120.0      1
1455435.0      1
1484705.0      1
Name: fnlwgt, Length: 21581, dtype: int64

In [104]:
K_foldCV = K_foldCV.drop(['fnlwgt'] , axis=1)
K_foldCV.head(5)

,age,workclass,education_num_x,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,education_num_y
0,39.0,State-gov,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,5013.0,0.0,40.0,United-States,<=50K,13.0
1,50.0,Self-emp-not-inc,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,13.0
2,38.0,Private,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,9.0
3,53.0,Private,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,7.0
4,28.0,Private,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,13.0


## 類別型資料進行dummy code

In [105]:
K_foldCV["income"] = K_foldCV["income"].replace(" <=50K","0")
K_foldCV["income"] = K_foldCV["income"].replace(" >50K","1")

In [106]:
K_foldCV["sex"] = K_foldCV["sex"].replace(" Male","1")
K_foldCV["sex"] = K_foldCV["sex"].replace(" Female","0")

In [107]:
K_foldCV.head(3)

,age,workclass,education_num_x,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,education_num_y
0,39.0,State-gov,13.0,Never-married,Adm-clerical,Not-in-family,White,1,5013.0,0.0,40.0,United-States,0,13.0
1,50.0,Self-emp-not-inc,13.0,Married-civ-spouse,Exec-managerial,Husband,White,1,0.0,0.0,13.0,United-States,0,13.0
2,38.0,Private,9.0,Divorced,Handlers-cleaners,Not-in-family,White,1,0.0,0.0,40.0,United-States,0,9.0


In [108]:
K_dummy=pd.get_dummies(K_foldCV,columns=['workclass','marital_status','occupation','relationship','race','native_country'])

In [109]:
K_dummy.head(3)

,age,education_num_x,sex,capital_gain,capital_loss,hours_per_week,income,education_num_y,workclass_5013.0,workclass_ Federal-gov,...,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia,native_country_None
0,39.0,13.0,1,5013.0,0.0,40.0,0,13.0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50.0,13.0,1,0.0,0.0,13.0,0,13.0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,38.0,9.0,1,0.0,0.0,40.0,0,9.0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [110]:
x = K_dummy.drop('income',axis=1)
y = K_dummy['income'] 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.70, random_state=5)
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
# 種類 2 種以下的類別型欄位轉標籤編碼 (Label Encoding)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# 檢查每一個 column
for col in K_foldCV:
    if K_foldCV[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(K_foldCV[col].unique())) <= 2:
            # 就做 Label Encoder
            le.fit(K_foldCV[col])
            K_foldCV[col] = le.transform(K_foldCV[col])
            
            
            # 紀錄有多少個 columns 被標籤編碼過
            le_count += 1
            
# 標籤編碼 (2種類別) 欄位轉 One Hot Encoding            
K_dummy= pd.get_dummies(K_foldCV)


In [ ]:
K_dummy.head(3)

In [ ]:
x = K_dummy.drop('income',axis =1)
y = K_dummy['income']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
clf = RandomForestClassifier(n_estimators=20, max_depth=4)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

## k fold

In [ ]:
def K_fold_CV(k, data):
    folds = []
    b = np.array([])
    for i in range(k):
        folds.append(np.array_split(K_dummy, k))
    for i in range (k):
        test_set = folds[i].pop(i)
        # testing
        X_test = test_set.drop("income", axis=1) 
        Y_test = test_set["income"]
        # training
        train = pd.concat(folds[i])
        X_train = train.drop("income", axis=1) 
        Y_train = train["income"]  

        rf = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=None)
        clf.fit(X_train,Y_train)
        Y_pred = clf.predict( X_test)
        b = np.append(b,[metrics.accuracy_score(Y_test, Y_pred)])
        print ("accuracy_score:",b)
    return b.mean()


K_fold_CV(10, K_dummy)
